In [32]:
import itertools
import h5py
import numpy as np
import pandas as pd
from einops import rearrange
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transformers
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [33]:
train_codes = pd.read_csv('train_embbeds.csv')
train_labels = pd.read_csv('train_labels.csv')
print(train_codes.shape)
print(train_labels.shape)

(53999, 1024)
(53999, 1)


In [34]:
train_set = train_codes.iloc[:,0:1024].values
train_set_labels= train_labels.iloc[:,:].values
print(train_set.shape)
print(train_set_labels.shape)

(53999, 1024)
(53999, 1)


In [35]:
test_codes = pd.read_csv('test_embbeds.csv')
test_labels = pd.read_csv('test_labels.csv')
print(test_codes.shape)
print(test_labels.shape)

(17999, 1024)
(17999, 1)


In [36]:
test_set = test_codes.iloc[:,0:1024].values
test_set_labels= test_labels.iloc[:,:].values
print(test_set.shape)
print(test_set_labels.shape)

(17999, 1024)
(17999, 1)


In [37]:
def sliding_windows(dataX,dataY, seq_length):
    print("shape of data: ", dataX.shape)
    print("shape of data labels: ", dataY.shape)
    print("seq_length: ", seq_length)
    
    #batch_len=len(dataX)
    
    #x = torch.empty((1,4,1024))
    # y = torch.empty((1,1))
    #y = torch.empty(1,1)
    
    x = []
    y = []
    
    for i in range(len(dataX)-seq_length-1):
        _x = dataX[i:(i+seq_length)]
        _y = dataY[i+seq_length]
        x.append(_x)
        y.append(_y)
    
    #for i in range(batch_len-seq_length-1):
        # _x = data.detach()[i:(i+seq_length)]
        # _y = data.detach()[i+seq_length]
        #_x = dataX[i:(i+seq_length)]
        #_y = dataY[i+seq_length]
        
        #_xOut=rearrange(_x,'w h -> 1 w h')
        #_yOut=rearrange(_y,'w -> 1 w')


        #x = torch.cat((x, _xOut),dim=0)
        #y = torch.cat((y, _yOut),dim=0)

    # return torch.tensor(x),torch.tensor(y)
    #print("Size of x: ", x.size())
    #print("Size of _x: ", _x.size())
    #print("Size of y: ", y.size())
    #print("Size of _y: ", _y.size())
    
    x_array=np.array(x)
    y_array=np.array(y)
    
    return torch.tensor(x_array), torch.tensor(y_array)

In [38]:
seq_length = 3
x_train, y_train = sliding_windows(train_set, train_set_labels, seq_length)

print("The shape of x_train: ", x_train.size())
print("The shape of y_train: ", y_train.size())

x_train=x_train.float()
y_train=y_train.float()

shape of data:  (53999, 1024)
shape of data labels:  (53999, 1)
seq_length:  3
The shape of x_train:  torch.Size([53995, 3, 1024])
The shape of y_train:  torch.Size([53995, 1])


In [39]:
seq_length = 3
x_test, y_test = sliding_windows(test_set, test_set_labels, seq_length)

print("The shape of x_train: ", x_test.size())
print("The shape of y_train: ", y_test.size())

x_test=x_test.float()
y_test=y_test.float() 

shape of data:  (17999, 1024)
shape of data labels:  (17999, 1)
seq_length:  3
The shape of x_train:  torch.Size([17995, 3, 1024])
The shape of y_train:  torch.Size([17995, 1])


In [40]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, device):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [41]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [42]:
trainX = Variable(x_train)
trainY = Variable(y_train)
    
print("Shape of trainX: ", trainX.size())
print("Shape of trainY: ", trainY.size())

Shape of trainX:  torch.Size([53995, 3, 1024])
Shape of trainY:  torch.Size([53995, 1])


In [43]:
testX = Variable(x_test)
testY = Variable(y_test)
    
print("Shape of trainX: ", testX.size())
print("Shape of trainY: ", testY.size())

Shape of trainX:  torch.Size([17995, 3, 1024])
Shape of trainY:  torch.Size([17995, 1])


In [44]:
num_epochs = 10
learning_rate = 0.001

input_size = 1024
hidden_size = 512
num_layers = 1

num_classes = 1

# locate our GPU device and initialize it for training
#use_cuda = torch.cuda.is_available()
#device = torch.device('cuda' if use_cuda else 'cpu')
device = torch.device('cpu')

lstm = LSTM(num_classes, input_size, hidden_size, num_layers, device)
lstm.to(device)
print("Number of parameters: ", sum(p.numel() for p in lstm.parameters()))

criterion = torch.nn.MSELoss()    # mean-sqaured error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    
    #Run a train on the train sequence
    lstm.train()
    outputs = lstm(trainX.to(device))
    optimizer.zero_grad()
    loss = criterion(outputs, trainY.to(device))
    loss.backward()
    optimizer.step()
    
    #Evaluate the epoch on the test sequence.
    lstm.eval()
    test_outputs = lstm(testX.to(device))
    test_loss = criterion(test_outputs, testY.to(device))
    
    if epoch % 2 == 0:
      print(f'Epoch {epoch}, Loss_train: {loss:.4f}, Loss_test: {test_loss:.4f}')

Number of parameters:  3150337
Epoch 0, Loss_train: 0.3515, Loss_test: 1.5097
Epoch 2, Loss_train: 0.4144, Loss_test: 0.0628
Epoch 4, Loss_train: 0.1309, Loss_test: 0.2176
Epoch 6, Loss_train: 0.1915, Loss_test: 0.1503
Epoch 8, Loss_train: 0.1068, Loss_test: 0.0831
